In [1]:
import sys
sys.path.append('..')
# reload local packages automatically
%load_ext autoreload
%autoreload 2

from jax import config
config.update("jax_enable_x64", True)

# Import packages.
import cvxpy as cp
import numpy as np

import matplotlib.pyplot as plt


from opentn.transformations import create_kitaev_liouvillians, exp_operator_dt, factorize_psd, super2choi
d, N = 2, 4
dim = d**N
Lvec, Lvec_odd, Lvec_even, Lnn = create_kitaev_liouvillians(N=N, d=d, gamma=1e-2)
tau = 4
superops_exp = []
for i, op in enumerate([Lvec, Lvec_odd, Lvec_even]):
    if i == 1:
        superops_exp.append(exp_operator_dt(op, tau/2, 'jax'))
    else:
        superops_exp.append(exp_operator_dt(op, tau, 'jax'))
exp_Lvec, exp_Lvec_odd, exp_Lvec_even = superops_exp

X1 = factorize_psd(psd=super2choi(exp_Lvec_odd), tol=1e-12)
X2 = factorize_psd(psd=super2choi(exp_Lvec_even), tol=1e-12)

from opentn.optimization import gds, frobenius_norm, model_Ys, compute_loss
cost_n4, grads_n4, xi_n4 = gds(fn=compute_loss, x0=[X1, X2, X1], iter=int(2e3), loss_fn=frobenius_norm, model=model_Ys, rate=1.5e-9, exact = exp_Lvec, show_cost=False)
print(cost_n4[0])
print(cost_n4[-1])

2.6383007149418305e-07
1.4977787039695335e-07


In [91]:
from opentn.transformations import super2choi, choi_composition
from opentn.optimization import small2zero
from scipy import sparse

with open('x_scs_warm_gds_eps_e-11.npy', 'rb') as f:
    Y1_cvxpy = np.load(f)
# rest of matrices optimized with GSD
X1_gds, X2_gds, X3_gds = xi_n4[-1]
Y1_gds = X1_gds@X1_gds.conj().T
Y2_gds = X2_gds@X2_gds.conj().T
Y3_gds = X3_gds@X3_gds.conj().T

tol = 1e-10

# rhs to which we want to approximate
rhs = np.asarray(super2choi(superop=exp_Lvec, dim=dim))
# srhs = sparse.csr_matrix(rhs) # small2zero was deleted
srhs = sparse.csr_matrix(small2zero(rhs, tol=tol)) # small2zero was deleted
srhs = srhs.astype(np.float64)
# change partial transpose to A
A = choi_composition(C1=small2zero(Y2_gds, tol=tol), C2=small2zero(Y3_gds, tol=tol), dim=dim) # small2zero was deleted
print(frobenius_norm(choi_composition(small2zero(Y1_cvxpy, tol=tol), A), small2zero(rhs, tol=tol))) # value when small2zero was used on A and rhs: 2.6522855638959974e-07
# update: apply small2zero to both A and rhs with same tol=1e-10

9.516735356249845e-08


/opt/miniconda3/envs/thesis/lib/python3.10/site-packages/scipy/sparse/_data.py:72: ComplexWarning: Casting complex values to real discards the imaginary part
  self.data.astype(dtype, casting=casting, copy=True),


In [92]:
print(frobenius_norm(choi_composition(small2zero(Y1_gds, tol=tol), small2zero(A, tol=tol)), small2zero(rhs, tol=tol))) 
print(cost_n4[-1])

1.4981540520646326e-07
1.4977787039695335e-07


In [94]:
for op in [Y1_gds, Y2_gds, Y3_gds, rhs]:
    print(sparse.csr_matrix(op).getnnz(), sparse.csr_matrix(small2zero(op, tol=tol)).getnnz())

19044 2980
19044 2592
19044 2976
3808 3152


In [57]:
frobenius_norm(link_product_cvxpy(Y1_cvxpy, A, dim=dim, transpose=0, simplify=True, tol=1e-10, optimization=False).value, small2zero(rhs, tol=1e-10))

Array(9.51436397e-08, dtype=float64)

In [71]:
print(frobenius_norm(rhs, small2zero(rhs, tol=tol)))
print(frobenius_norm(rhs, small2zero(rhs, tol=1e-10)))

1.0853149403921307e-10
1.4575489056804186e-10


In [80]:
print(np.linalg.matrix_rank(rhs, tol=tol))
print(np.linalg.matrix_rank(small2zero(rhs, tol=tol), tol=tol))

16
17


In [82]:
max_idx = 20
print(np.sort(np.linalg.svd(rhs, compute_uv=False))[::-1][:max_idx])
print('----')
print(np.sort(np.linalg.svd(small2zero(rhs, tol=tol), compute_uv=False))[::-1][:max_idx])

[1.58809071e+01 5.35571431e-02 3.96274662e-02 2.56236967e-02
 1.19766269e-04 4.95029797e-05 4.94826258e-05 4.08201317e-05
 2.47515434e-05 8.95308390e-08 7.94678759e-08 5.63855416e-08
 4.35119228e-08 1.83024667e-08 1.61593461e-08 8.50200640e-09
 9.97181402e-11 6.72504453e-11 3.81643688e-11 3.10066063e-11]
----
[1.58809071e+01 5.35571431e-02 3.96274662e-02 2.56236967e-02
 1.19766270e-04 4.95029806e-05 4.94826266e-05 4.08201301e-05
 2.47515418e-05 8.95279320e-08 7.94676789e-08 5.63870560e-08
 4.35160724e-08 1.83035080e-08 1.61580364e-08 8.50463023e-09
 1.00970459e-10 8.72534489e-11 5.47195524e-11 4.95503676e-11]


In [24]:
frobenius_norm(link_product_cvxpy(Y1_cvxpy, A, dim=dim, transpose=1, simplify=True).value, rhs)

/opt/miniconda3/envs/thesis/lib/python3.10/site-packages/scipy/sparse/_data.py:72: ComplexWarning: Casting complex values to real discards the imaginary part
  self.data.astype(dtype, casting=casting, copy=True),


Array(1.56258576e-07, dtype=float64)

In [90]:
# now lets try an optimization for the third layer using as a warm start the solution that we found with GDS as well.
# now C1 = Y2 @ Y1 and C2 = Y3_opt
from opentn.transformations import link_product_cvxpy
Y3_cvxpy = cp.Variable((dim**2,dim**2), PSD=True)
lhs = link_product_cvxpy(C1=small2zero(choi_composition(Y1_cvxpy, Y2_gds), tol=tol), C2=Y3_cvxpy, dim=dim, transpose=0, optimization=True)
cost = cp.norm(lhs - srhs, "fro")
prob = cp.Problem(cp.Minimize(cost))
Y3_cvxpy.value = small2zero(Y3_gds, tol=tol).astype(np.float64) 
cost.value

/var/folders/lv/986s3mp97gv9n3rb757bg87m0000gn/T/ipykernel_4057/1132691232.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  Y3_cvxpy.value = small2zero(Y3_gds, tol=tol).astype(np.float64)


9.516185192969064e-08

In [35]:
test_Y3_cvxpy = cp.Variable((dim**2,dim**2), PSD=True)
test_Y3_cvxpy.value = small2zero(Y3_gds, tol=1e-11).astype(np.float64)
test_lhs = link_product_cvxpy(C1=choi_composition(Y1_cvxpy, Y2_gds, dim=dim), C2=test_Y3_cvxpy, dim=dim, transpose=0, simplify=True, tol=1e-11, optimization=True)
print(frobenius_norm(test_lhs.value, rhs)) # value when small2zero was used on A and rhs: 2.6522855638959974e-07

/var/folders/lv/986s3mp97gv9n3rb757bg87m0000gn/T/ipykernel_4057/4238069078.py:2: ComplexWarning: Casting complex values to real discards the imaginary part
  test_Y3_cvxpy.value = small2zero(Y3_gds, tol=1e-11).astype(np.float64)


9.519125511238636e-08


In [51]:
prob.solve(solver=cp.SCS, verbose=True, canon_backend=cp.SCIPY_CANON_BACKEND, eps=0.25e-10)

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Jul 27 01:33:02 PM: Your problem has 65536 variables, 0 constraints, and 0 parameters.
(CVXPY) Jul 27 01:33:02 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 27 01:33:02 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 27 01:33:02 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 27 01:33:02 PM: Compiling problem (target solver=SCS).
(CVXPY) Jul 27 01:33:02 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing 

9.518920490166243e-08

In [40]:
# this si the one where the small2zero was not apply everywhere
prob.solve(solver=cp.SCS, verbose=True, canon_backend=cp.SCIPY_CANON_BACKEND, eps=0.25e-10)

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Jul 26 02:12:35 PM: Your problem has 65536 variables, 0 constraints, and 0 parameters.
(CVXPY) Jul 26 02:12:35 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 26 02:12:35 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 26 02:12:35 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 26 02:12:35 PM: Using cached ASA map, for faster compilation (bypassing reduction chain).
(CVXPY) Jul 26 02:12:35 PM: Finished problem compilation (t

9.518053924910776e-08

In [84]:
for op in [Y3_cvxpy.value, Y3_gds]:
    print(np.linalg.matrix_rank(op, tol=tol))

12
4


In [ ]:
# eps: 5e-10: 9.517969895842572e-08
# eps: 1e-10: 9.518041680165088e-08
# eps: 0.5e-10: 9.517904309118482e-08
